# Compile

The need to compile the MicroPython interpreter arises frequently, for example to ["freeze" pre-compiled Python code](https://docs.micropython.org/en/latest/reference/packages.html?highlight=freeze) into the interpreter or [add new features](https://docs.micropython.org/en/latest/develop/extendingmicropython.html).

Below we briefly outline the steps for compiling MicroPython in *ide49*.

**Note:** At the time of writing, compiling for the ESP32 is is [not yet available](https://github.com/espressif/esp-idf/issues/6432) for 64-Bit ARM CPUs (Raspberry PI). 

## Clone MicroPython

Download from github or update an existing copy.

In [1]:
%%bash

# $IOT/mp is ide49's default folder for MicroPython source code and customizations
mkdir -p $IOT/mp

cd $IOT/mp
if [ ! -d micropython ]; then
    # clone the repo
    git clone https://github.com/micropython/micropython.git
else
    # update a previously downloaded repo
    cd micropython
    git checkout master
    git pull
    git merge master
fi

Already on 'master'
Your branch is up to date with 'origin/master'.
From github.com:micropython/micropython
   e2ca8ab8f..bb7aae557  master     -> origin/master
Updating e2ca8ab8f..bb7aae557
Fast-forward
 ports/stm32/boards/NUCLEO_WB55/mpconfigboard.mk           |  7 -------
 ports/stm32/boards/STM32F769DISC/mpconfigboard.mk         |  3 ---
 tools/autobuild/autobuild.sh                              |  7 ++++---
 tools/autobuild/build-boards.sh                           | 15 ++++++++++-----
 .../{build-stm32-latest.sh => build-stm32-extra.sh}       | 13 ++-----------
 tools/ci.sh                                               |  8 ++++++--
 6 files changed, 22 insertions(+), 31 deletions(-)
 rename tools/autobuild/{build-stm32-latest.sh => build-stm32-extra.sh} (82%)
Already up to date.


## mpy-cross

Follow the instructions at [MicroPython github repository](https://github.com/micropython/micropython). The first step is to compile the MicroPython cross-compiler. 

**Note:** The compiler is installed in a separate container, `arm32`, accessed via the the `%%service` magic. Alternatively you can login to the [arm32 service](/arm32) at http://iot49.local/arm32 and open a terminal to run commands.

In [1]:
%%service arm32

cd $IOT/mp/micropython/mpy-cross
make

README.md
bin
ide49
iot-device
iot-kernel
iot49-dev
iot49.org
micropython
micropython_github
micropython_modules
scratch
Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
mkdir -p build/genhdr
GEN build/genhdr/mpversion.h
GEN build/genhdr/moduledefs.h
GEN build/genhdr/qstr.i.last
GEN build/genhdr/qstr.split
GEN build/genhdr/qstrdefs.collected.h
QSTR updated
GEN build/genhdr/qstrdefs.generated.h
mkdir -p build/py/
mkdir -p build/shared/runtime/
CC ../py/mpstate.c
CC ../py/nlr.c
CC ../py/nlrx86.c
CC ../py/nlrx64.c
CC ../py/nlrthumb.c
CC ../py/nlraarch64.c
CC ../py/nlrpowerpc.c
CC ../py/nlrxtensa.c
CC ../py/nlrsetjmp.c
CC ../py/malloc.c
CC ../py/gc.c
CC ../py/pystack.c
CC ../py/qstr.c
CC ../py/vstr.c
CC ../py/mpprint.c
CC ../py/unicode.c
CC ../py/mpz.c
CC ../py/reader.c
CC ../py/lexer.c
CC ../py/parse.c
CC ../py/scope.c
CC ../py/compile.c
CC ../py/emitcommon.c
CC ../py/emitbc.c
CC ../py/asmbase.c
CC ../py/asmx64.c
CC ../py/emitnx64.c
CC ../py/asmx86.c
CC .

## ARM Ports

The example below is for the *stm32* port. The procedure for other ports is similar, follow the instructions for the particular port on the [MicroPython github repository](https://github.com/micropython/micropython).

The arm32 cross compiler is installed in container with the same name. Access it from the IoT Kernel using the `%%service` magic.

In [ ]:
%%service arm32

# submodules
cd $IOT/mp/micropython/ports/stm32
git submodule update --init

In the commands below, replace `PYBV11` with the name of your configuration.

In [1]:
%%service arm32

# compile
cd $IOT/mp/mp/micropython/ports/stm32
make clean BOARD=PYBV11
make BOARD=PYBV11

Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
rm -rf build-PYBV11 
Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
mkdir -p build-PYBV11/genhdr
GEN build-PYBV11/genhdr/pins.h
GEN stmconst build-PYBV11/modstm_qstr.h
GEN build-PYBV11/genhdr/mpversion.h
GEN build-PYBV11/genhdr/moduledefs.h
GEN build-PYBV11/genhdr/pybcdc.inf
GEN build-PYBV11/genhdr/pybcdc_inf.h
GEN build-PYBV11/genhdr/pllfreqtable.h
GEN build-PYBV11/genhdr/qstr.i.last
GEN build-PYBV11/genhdr/qstr.split
GEN build-PYBV11/genhdr/qstrdefs.collected.h
QSTR updated
GEN build-PYBV11/genhdr/qstrdefs.generated.h
GEN build-PYBV11/genhdr/compressed.split
GEN build-PYBV11/genhdr/compressed.collected
Compressed data updated
GEN build-PYBV11/genhdr/compressed.data.h
mkdir -p build-PYBV11/build-PYBV11/
mkdir -p build-PYBV11/drivers/bus/
mkdir -p build-PYBV11/drivers/dht/
mkdir -p build-PYBV11/drivers/memory/
mkdir -p build-PYBV11/extmod/
mkdir -p build-PYBV11/lib/emb

## ESP32 Port

The [esp-idf]() compiler is installed in container with the same name. You can also get a [console](/esp-idf) at http://iot49.local/esp-idf.

In [1]:
%%service esp-idf

# submodules
cd $IOT/mp/mp/micropython/ports/esp32
make submodules

setting up IDF ...
git submodule update --init ../../lib/berkeley-db-1.xx


In [1]:
%%service esp-idf

# compile
cd $IOT/mp/mp/micropython/ports/esp32
make BOARD=GENERIC_OTA

setting up IDF ...
idf.py -D MICROPY_BOARD=GENERIC_OTA -B build-GENERIC_OTA  build
[1/203] cd /home/iot/micropython/ports/esp32/build-GENERIC_OTA/esp-idf/main && echo -n
[2/4] Performing build step for 'bootloader'
ninja: no work to do.
Executing action: all (aliases: build)
Running ninja in directory /home/iot/micropython/ports/esp32/build-GENERIC_OTA
Executing "ninja all"...

Project build complete. To flash, run this command:
/opt/esp/python_env/idf4.2_py3.7_env/bin/python ../../../../../opt/esp/idf/components/esptool_py/esptool/esptool.py -p (PORT) -b 460800 --before default_reset --after hard_reset --chip esp32  write_flash --flash_mode dio --flash_size detect --flash_freq 40m 0x1000 build-GENERIC_OTA/bootloader/bootloader.bin 0x8000 build-GENERIC_OTA/partition_table/partition-table.bin 0xd000 build-GENERIC_OTA/ota_data_initial.bin 0x10000 build-GENERIC_OTA/micropython.bin
or run 'idf.py -p (PORT) flash'
bootloader  @0x001000    22720  (    5952 remaining)
partitions  @0x008000   

Find port ...

In [1]:
%discover -a

/dev/ttyUSB0
  hwid         USB VID:PID=10C4:EA60 SER=0160B5B8 LOCATION=1-5.2
  manufacturer Silicon Labs
  product      CP2104 USB to UART Bridge Controller
  interface    CP2104 USB to UART Bridge Controller



Update the port to match the output from the previous cell (e.g. /dev/ttyUSB0), then run the next cell to flash the code to the microcontroller.

In [1]:
%%service esp-idf

# flash
cd $IOT/mp/mp/micropython/ports/esp32
make BOARD=GENERIC_OTA PORT=/dev/ttyUSB0 deploy

setting up IDF ...
idf.py -D MICROPY_BOARD=GENERIC_OTA -B build-GENERIC_OTA  -p /dev/ttyUSB0 -b 460800 flash
[1/204] cd /home/iot/micropython/ports/esp32/build-GENERIC_OTA/esp-idf/main && echo -n
[2/5] Performing build step for 'bootloader'
ninja: no work to do.
[2/3] cd /opt/esp/idf/components/esptool_py && /opt/esp/tools/cmake/3.16.4/bin/cmake -D IDF_PATH="/opt/esp/idf" -D ESPTOOLPY="/opt/esp/python_env/idf4.2_py3.7_env/bin/python /opt/esp/idf/components/esptool_py/esptool/esptool.py --chip esp32" -D ESPTOOL_ARGS="--before=default_reset --after=hard_reset write_flash @flash_args" -D WORKING_DIRECTORY="/home/iot/micropython/ports/esp32/build-GENERIC_OTA" -P /opt/esp/idf/components/esptool_py/run_esptool.cmake
esptool.py --chip esp32 -p /dev/ttyUSB0 -b 460800 --before=default_reset --after=hard_reset write_flash --flash_mode dio --flash_freq 40m --flash_size 4MB 0x8000 partition_table/partition-table.bin 0xd000 ota_data_initial.bin 0x1000 bootloader/bootloader.bin 0x10000 micropython.b

Verify it's working ...

In [1]:
%discover 
%connect esp32

print("installed modules ...")
help('modules')

esp32  serial:///dev/ttyUSB0  
Connected to esp32 @ serial:///dev/ttyUSB0
installed modules ...
__main__          framebuf          uasyncio/lock     urandom
_boot             gc                uasyncio/stream   ure
_onewire          inisetup          ubinascii         uselect
_thread           machine           ubluetooth        usocket
_uasyncio         math              ucollections      ussl
_webrepl          micropython       ucryptolib        ustruct
apa106            neopixel          uctypes           usys
btree             network           uerrno            utime
builtins          ntptime           uhashlib          utimeq
cmath             onewire           uheapq            uwebsocket
dht               uarray            uio               uzlib
ds18x20           uasyncio/__init__ ujson             webrepl
esp               uasyncio/core     uos               webrepl_setup
esp32             uasyncio/event    upip              websocket_helper
flashbdev         uasyncio/funcs 

## Unix Port

In [ ]:
%%service arm32

cd $IOT/mp/mp/micropython/ports/unix
make submodules
make

Check it out:

In [1]:
%%bash

cd $IOT/mp/mp/micropython/ports/unix
./micropython -c "import sys; print(sys.platform)"

linux


In [1]:
%%bash

cd $IOT/mp/mp/micropython/ports/unix
./micropython -m upip install micropython-pystone
./micropython -m pystone

Installing to: /home/iot/.micropython/lib/
Installing micropython-pystone 3.4.2.post2 from https://micropython.org/pi/pystone/pystone-3.4.2.post2.tar.gz
Pystone(1.2) time for 50000 passes = 0.981
This machine benchmarks at 50968.4 pystones/second
